# Improve Email Engagement

# STEP 1 : Feature Extraction

In [2]:
from typing import List,Dict,Optional
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage,SystemMessage
import base64
from langchain_openai import OpenAI
from io import BytesIO
import os
import pandas as pd
from datetime import date
import openai
from PIL import Image
import base64
from io import BytesIO

/Users/ujjaini.dalal/Library/Python/3.13/lib/python/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# API_KEY
api_key = "ghjkl;'"

In [4]:
# Function to convert PIL image to base64
def pil_image_to_base64(pil_image):
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

# Function to convert image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


# system prompt Text
system_prompt = """
Extract the following details from the provided image:

1. File Name: Put the exact file name from image path.
2. Subject: Extract the subject exactly as displayed.
3. Subject Length : Extract the length of the subject in characters.
4. Email Tone : What is the tone of the emial? Is it Promotional, Informative or Convincing if not then set to Others?
5. Email Type : What is the type of the email? Is it a Transactional, Survey, Informational ,Newsletter if not then set to Others?
6. Body Tone : What is the tone of the body? Is it Friendy, Convincing,Inspirational or Professional if not then set to Others?
7. Body Type : What is the body type? Is it a Reminder, Announcement,Transactional or Awareness if not then set to Others?
8. Body Length : Extract the length of the body in characters.
9. Campaign Body : Extract the body of the email.
10. Offer Flag : If the image contains an offer, if its a discount offer/ free item offer then set the flag to True, else False. 
11. Offer Type: What is the offer? Is it a discount offer/ free item offer else set as No Offer?
12. Offer : What is the offer? Extract the exact offer, or set to No Offer if not specified.
13. Offer Value: Extract the exact offer value, or set to zero if not specified.It should be in percentage or amount.
14. CTA Order Now: If the Call to Action says order now, set the flag to True, else False.
15. CTA Visit Chilis : If the Call to Action says visit Chilis, set the flag to True, else False.
16. Image Flag : If the file contains an image, set the flag to True, else False.
17. Image : Extract what does the image is about? Is it a food item or beverage?
18. Image_Item : Extract the item in the image. If the image is about a food item, extract the food item name.
19. Color_Red_Themed : Whether the email is themed in red color or not. If yes then set the flag to True, else False.
"""

In [ ]:
# JANUARY

# Folder path containing the PNG images
folder_path = '01 - January/'
# List all files in the folder and filter out only the .png files
image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.png')]


# Initialize an empty list to store the responses
responses = []

# Loop through each image path
for image_path in image_paths:
    base64_image = encode_image(image_path)
    image_file_name = os.path.basename(image_path)
    # print(f"Processing image: {image_file_name}") 

    # Update the user prompt with the new image
    user_prompt = [
        {"type": "image_url",
         "image_url": {
             "url": f"data:image/jpeg;base64,{base64_image}"
         }}
    ]

    # Update the file name in the FetchEmailDetails model
    class FetchEmailDetails(BaseModel):
        file_name:str 
        subject: str = Field(description="Email Subject", default="")
        subject_length: int = Field(description="Length of the Subject in characters", default=0)
        email_tone: str = Field(description="Email Tone", default="")
        email_type: str = Field(description="Email Type", default="")
        body_tone: str = Field(description="Body Tone", default="")
        body_type: str = Field(description="Body Type", default="")
        body_length: int = Field(description="Length of the Body in characters", default=0)
        campaign_body: str = Field(description="Campaign Body", default="")
        offer_flag: bool = Field(description="Offer Flag", default=False)
        offer_type: str = Field(description="Offer Type", default="")
        offer: str = Field(description="Offer", default="")
        # image_offer: str = Field(description="Image Offer", default="")
        # offer_expiry_date: date = Field(description="Offer Expiry Date in YYYY-MM-DD format", default=date(1, 1, 1))
        offer_value: str = Field(description="Offer Value", default=0)
        # points_awarded: int = Field(description="Points Awarded", default=0)
        cta_order_now: bool = Field(description="CTA Order Now Flag", default=False)
        cta_visit_chilis: bool = Field(description="CTA Visit Chilis Flag", default="")
        image_flag: bool = Field(description="Image Flag", default=False)
        image : str = Field(description="Image", default="")
        image_item : str = Field(description="Image Item", default="")
        color_theme:bool = Field(description="Color theme", default="")
        # gif_flag: bool = Field(description="GIF Flag", default=False)

    llm = ChatOpenAI(**{
        'temperature': 0,
        'api_key': api_key,
        'base_url': 'https://relay.ai.gale.technology/api/relay/openai/v1',
        'model': 'gpt-4o',
        'max_tokens': 4000,
    })

    structured_llm = llm.with_structured_output(FetchEmailDetails)

    # Invoke the model
    response = structured_llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)])
    
    # Update the File Name 
    response.file_name = image_file_name
    
    # Convet fields to uppercase
    response.subject = response.subject.upper()
    response.email_type = response.email_type.upper()
    response.email_tone = response.email_tone.upper()
    response.body_type = response.body_type.upper()
    response.body_tone = response.body_tone.upper()
    response.image = response.image.upper()
    response.image_item = response.image_item.upper()
    response.offer_type = response.offer_type.upper()

    # Append the response to the list
    responses.append(response.dict())


# Convert the list of responses to a DataFrame
df_responses2 = pd.DataFrame(responses)
df_responses2

,file_name,subject,subject_length,email_tone,email_type,body_tone,body_type,body_length,campaign_body,offer_flag,offer_type,offer,offer_value,cta_order_now,cta_visit_chilis,image_flag,image,image_item,color_theme
0,BCRA_202401_02_AWARE_3FORME_0105.png,THE FORMULA FOR YOUR PERFECT MEAL,34,CONVINCING,PROMOTIONAL,CONVINCING,ANNOUNCEMENT,207,"Sick of cooking, but still want a filling, fla...",True,DISCOUNT OFFER,3 for Me starting at $10.99,$10.99,True,False,True,FOOD ITEM,CHIPS & SALSA,True
1,BCRA_202401_11_MARG_SURVEY_0130.png,HIT US BACK WITH FEEDBACK,25,INFORMATIVE,SURVEY,FRIENDLY,SURVEY,276,We hope you enjoy your Chili’s favorites as mu...,False,NO OFFER,No Offer,0,False,False,True,FOOD ITEM,BURGER AND FRIES,True
2,BCRA_202401_01_ALC_MOTM_0102_V1.png,A MARG FOR EVERY MOOD,20,INFORMATIVE,NEWSLETTER,INFORMATIVE,ANNOUNCEMENT,445,"The new year is here, which means it’s time fo...",True,DISCOUNT OFFER,$6 Spice-A-Rita and $6 Nice-A-Rita,$6,False,False,True,BEVERAGE,SPICE-A-RITA AND NICE-A-RITA,True
3,BCRA_202401_07_ALC_HH_0121_V1.png,GAME DAY IS ON AT CHILI’S,23,INFORMATIVE,INFORMATIONAL,CONVINCING,ANNOUNCEMENT,207,You really can’t beat Sunday football at Chili...,True,DISCOUNT OFFER,$4 Modelos during Happy Hour,4,False,False,True,BEVERAGE,MODELO,True
4,BCRA_202401_07_ALC_HH_0121_V3.png,GAME DAY IS ON AT CHILI'S,23,INFORMATIVE,INFORMATIONAL,CONVINCING,ANNOUNCEMENT,230,You really can’t beat Sunday football at Chili...,True,DISCOUNT OFFER,Happy Hour,0,False,False,True,BEVERAGE,MODELO,True
5,BCRA_202401_01_ALC_MOTM_0102_V2.png,A MARG FOR EVERY MOOD,20,INFORMATIVE,NEWSLETTER,INFORMATIVE,ANNOUNCEMENT,442,"The new year is here, which means it’s time fo...",False,NO OFFER,No Offer,0,False,False,True,BEVERAGE,SPICE-A-RITA AND NICE-A-RITA,True
6,BCRA_202401_07_ALC_HH_0121_V2.png,GAME DAY IS ON AT CHILI'S,23,INFORMATIVE,INFORMATIONAL,CONVINCING,ANNOUNCEMENT,230,You really can’t beat Sunday football at Chili...,True,DISCOUNT OFFER,2-for-1 Modelos® during Happy Hour,2-for-1,False,False,True,FOOD AND BEVERAGE,JUICY WINGS AND MODELOS®,True
7,BCRA_202401_06_ALC_CRISPERS_MOTM_0119_V2.png,THESE CHICKEN CRISPERS ARE BRINGING THE HEAT,45,INFORMATIVE,NEWSLETTER,INFORMATIVE,ANNOUNCEMENT,442,Warning: Dinner tonight might make you sweat. ...,False,NO OFFER,No Offer,0,True,False,True,FOOD ITEM,CHICKEN CRISPERS,True
8,BCRA_202401_06_ALC_CRISPERS_MOTM_0119_V1.png,THESE CHICKEN CRISPERS ARE BRINGING THE HEAT,45,INFORMATIVE,NEWSLETTER,INFORMATIVE,ANNOUNCEMENT,451,Warning: Dinner tonight might make you sweat. ...,True,DISCOUNT OFFER,$6 Spice-a-Rita,$6,True,False,True,FOOD ITEM,NASHVILLE HOT CHICKEN CRISPERS,True
9,BCRA_202401_08_AWARE_TO-GO-CRISPERS_0125.png,THESE CHICKEN TENDERS ARE CERTIFIED CRISPY,43,INFORMATIVE,INFORMATIONAL,INFORMATIVE,ANNOUNCEMENT,252,It's right there in the name — our signature C...,False,NO OFFER,No Offer,0,True,False,True,FOOD ITEM,CHICKEN TENDERS,True


# Combine all the DFs

In [39]:
file_list = [df_responses1, df_responses2, df_responses3, df_responses4, df_responses5, df_responses6, df_responses7, df_responses8, df_responses9, df_responses10, df_responses11]

# Initialize final_df
final_df = pd.DataFrame()

for file in file_list:
    print('Shape:', file.shape)
    final_df = pd.concat([final_df, file], axis=0)

print('Shape:', final_df.shape)

Shape: (18, 19)
Shape: (14, 19)
Shape: (32, 19)
Shape: (26, 19)
Shape: (26, 19)
Shape: (28, 19)
Shape: (30, 19)
Shape: (23, 19)
Shape: (27, 19)
Shape: (20, 19)
Shape: (34, 19)
Shape: (278, 19)


In [40]:
final_df['file_name'] = final_df['file_name'].str.replace('.png','')
final_df['file_name'].unique()

array(['BIJWA_202401_01_AWARE_VARIETY_0110',
       'BIJWA_202401_02_AWARE_10-OFF-25-PROMO_0120',
       'IJW_PRO_WIN__5off_20_ALL_na_241025_B',
       'IJW_PRO_WIN_10-OFF-25_ALL__240729_B',
       'IJW_PRO_WIN_$10off$25_ALL_na_240928_B',
       'IJW_PRO_WIN_$10off$25_ALL_na_240210_B',
       'IJW_PRO_WIN_NoOffer_ALL_na_240713_B',
       'RewardExpirationReminder_ALL_T3.3',
       'RewardExpirationReminder_ALL_T3.2',
       'BIJWA_202401_03_AWARE_5-OFF-25_0131',
       'IJW_PRO_WIN_$5off$20_ALL_na_240827_B', 'VisitChilisReminder_T2',
       'RewardExpirationReminder_ALL_T3.1',
       'IJW_PRO_WIN_NoOffer_ALL_na_241005_B',
       'IJW_PRO_WIN_LazySunday_ALL_na_240225_B', 'RewardRenewed_ALL_T1',
       'IJW_PRO_WIN_NoOffer_ALL_na_240810_B',
       'IJW_PRO_WIN_NoOffer_ALL_na_240908_B',
       'BCRA_202401_02_AWARE_3FORME_0105',
       'BCRA_202401_11_MARG_SURVEY_0130',
       'BCRA_202401_01_ALC_MOTM_0102_V1', 'BCRA_202401_07_ALC_HH_0121_V1',
       'BCRA_202401_07_ALC_HH_0121_V3', 'BCRA

In [ ]:
final_df.to_csv('File1_Combibed_Feature_Output_v2_20241202.csv',index=False)